In [1]:
!pip install tensorflow==1.13.1

In [2]:
import tensorflow as tf
import os
import time
import pickle
import numpy as np

print(tf.__version__)
CIFAR_DIR = '/kaggle/input/cifar10/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1
['data_batch_1', 'data_batch_2', 'batches.meta', 'test_batch', 'data_batch_3', 'data_batch_5', 'data_batch_4', 'readme.html']


In [3]:
train_filenames = [os.path.join(CIFAR_DIR, f'data_batch_{i}') for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
print(train_filenames)
print(test_filenames)   

['/kaggle/input/cifar10/cifar-10-batches-py/data_batch_1', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_2', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_3', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_4', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_5']
['/kaggle/input/cifar10/cifar-10-batches-py/test_batch']


In [4]:
def load_data(filename):
    """前面是图片特征，后面是图片分类标签"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']


# tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_lables = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)  # all_data[0]列表中10000个存有(32*32*3=)3072个数的一维列表c
            all_lables.append(labels)  # all_data[0]列表中10000个标签值
        self._data = np.vstack(all_data)  # 垂直方向沿轴2堆叠 变为ndarray
        self._data = self._data / 127.5 - 1  # 归一化到范围[-1,1]
        self._lables = np.hstack(all_lables)  # 一维列表变成ndarray
        print(self._data.shape)  # (50000, 3072) 每一张图像展平后的结果
        print(self._lables.shape)
        print(self._lables[:5])

        self._num_samples = self._data.shape[0]  # 样本数
        self._need_shuffle = need_shuffle  # 进行洗牌随机
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()

    def _shuffle_data(self):
        # 通过随机排列下标进行随机样本
        p = np.random.permutation(self._num_samples)  # 变成随机下标
        self._data = self._data[p]
        self._lables = self._lables[p]

    def next_batch(self, batch_size):
        """return batch_size samples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_samples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = self._indicator + batch_size
            else:
                raise Exception('have no more samples')
        if end_indicator > self._num_samples:
            # 去除不足batch_size剩余的样本
            raise Exception('batch size is larger than the remaining samples')
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._lables[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels


train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
[6 9 9 4 1]
(10000, 3072)
(10000,)
[3 8 8 0 6]


In [5]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
x_image = tf.reshape(x, [-1, 3, 32, 32])  # 原数据channels_first存储的
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])  # 功类似np.rollaxis轴滚动

In [6]:

conv1_1 = tf.layers.conv2d(x_image,
                           32,  # output channel number
                           (3, 3),  # kernel size
                           # 明确指定stride时padding='same'也不能保持形状不变 就像output_size等于input_size/stride
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv1_1')
conv1_2 = tf.layers.conv2d(conv1_1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv1_2')

pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name='pool1')
# pooling1输出形状 16 * 16 * 32
conv2_1 = tf.layers.conv2d(pooling1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv2_2')
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2),
                                   (2, 2),
                                   name='pool2')
# pooling2输出形状 8 * 8 * 32
conv3_1 = tf.layers.conv2d(pooling2,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv3_2')
# # pooling3输出形状 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name='pool3')
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)
# 深度学习中，logits就是最终的全连接层的输出
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# softmax(y_)->p->one_hot->loss=plog(p)
predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)  # y是真实值
# 计算张量维度上元素的平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
with tf.name_scope('train_op'):
    # 这里计算梯度并更新了梯度
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [9]:
!rm -rf run_vgg_tensorboard

In [10]:
# 输出对应的数据来绘制图形
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)  #均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  #标准差
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)  #直接把变量放进去看直方图


with tf.name_scope('summary'):
    # merge_all()
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)
source_image = (x_image + 1) * 127.5  #归一化的逆过程
inouts_summary = tf.summary.image('inputs_image', source_image)
merged_summary = tf.summary.merge_all()  # 训练集关注(各层)输入、loss、accuracy等所有添加到summary中的指标
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])  # 测试集只关注loss、accuracy

In [11]:
LOG_DIR = '.'
run_label = './run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [14]:
init = tf.global_variables_initializer()  # 低版本tf必须要做
batch_size = 20
train_steps = 10000
test_steps = 100
output_summay_every_steps = 100  #每100次计算一次summary
with tf.Session() as s:
    s.run(init)  # 低版本tf必须要做
    start=time.time()
    train_writer = tf.summary.FileWriter(train_log_dir, s.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i + 1) % output_summay_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
        eval_ops_results = s.run(eval_ops, feed_dict={x: batch_data, y: batch_labels})
        loss_val, accu_val = eval_ops_results[0:2]  # [loss, accuracy, train_op，merged_summary]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i + 1)  # 写入文件
            test_summary_str = s.run([merged_summary_test],
                                     feed_dict={x: fixed_test_batch_data, y: fixed_test_batch_labels})[0]
            test_writer.add_summary(test_summary_str, i + 1)
            
        if (i + 1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, accuracy: %4.5f'
                  % (i + 1, loss_val, accu_val))
            end = time.time()
            print(f'just 100 steps take time: {end - start}s')
            start=end
        if (i + 1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)  # 重新拿测试集
            all_test_accu_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_accu_val = s.run([accuracy],
                                      feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_accu_val.append(test_accu_val)
            test_accu = np.mean(all_test_accu_val)
            print(f'now time={time.time()}')
            print('[Test ] Step: %d, accuracy: %4.5f' % (i + 1, test_accu))

[Train] Step: 100, loss: 2.15765, accuracy: 0.20000
just 100 steps take time: 15.139995574951172s
[Train] Step: 200, loss: 2.15854, accuracy: 0.20000
just 100 steps take time: 11.522160053253174s
[Train] Step: 300, loss: 1.78445, accuracy: 0.30000
just 100 steps take time: 11.595017671585083s
[Train] Step: 400, loss: 1.32181, accuracy: 0.65000
just 100 steps take time: 11.646500587463379s
[Train] Step: 500, loss: 1.27510, accuracy: 0.40000
just 100 steps take time: 12.217529296875s
[Train] Step: 600, loss: 1.61836, accuracy: 0.30000
just 100 steps take time: 11.280322313308716s
[Train] Step: 700, loss: 1.28683, accuracy: 0.50000
just 100 steps take time: 11.393040657043457s
[Train] Step: 800, loss: 1.06082, accuracy: 0.65000
just 100 steps take time: 12.238139152526855s
[Train] Step: 900, loss: 1.35736, accuracy: 0.60000
just 100 steps take time: 11.231126308441162s
[Train] Step: 1000, loss: 1.05223, accuracy: 0.45000
just 100 steps take time: 11.308213233947754s
(10000, 3072)
(10000,)